[← Control Systems as Dynamical Systems](../../../getting_started/theory_to_python/control_systems_as_dynamical_systems.rst)

# Example: Car Cruise Control

Suppose we want to build a robot. If we were hobbyists, we might want a weekend project; if we were engineers, we might need an extra arm on the assembly line; if we were academics, we might dream of a quadruped with a cannon-head to threaten the government for grant money.

Of course, we must begin somewhere. In this notebook, we take a classic example of a feedback system -- a car with cruise control -- and use pykal to take it from a block diagram to a composition of dynamical systems.

## System Overview

Suppose we are given car and are tasked with developing the control system behind a simple cruise control. We have at our disposal the acceleromoter (which will give us a measurment of our current speed), a button on the steering wheel (which will enable us to set the cruise control speed), and the car's CPU (with which we can implement any control algorithm we wish). With these objects, we may create a simple block diagram that represents our control system.

## Block Diagram
 Recall that the following is a block diagram for a simple feedback system:

![Traditional feedback system diagram](../../../_static/tutorial/theory_to_python/feedback_system_trad.svg)

where $r$ is the **setpoint** (or **reference point**),
$u$ is the **control input**,
$x$ is the (hidden) **state** of the plant,
$\hat{x}$ is the **state estimate**,
and $e$ is the **error** term.

In the case of our car-cruise control system, we can leverage this abstraction with a simple relabling:

![Traditional feedback system diagram](../../../_static/tutorial/theory_to_python/cruise_control_as_a_feedback_system.svg)

where the **speed setpoint** creats a desired speed signal, **PID** is a proportional-integrative-differential control algorithm, **KF** is a Kalman filter, and the **car** is, well, the car.
:::{note} If you were to ask five control theorists to design a block diagram, you would get six different diagrams. Don't fret the particulars; the diagram is simply a tool to organize our thinking.

### Discrete-time Dynamical Systems


For a minimal working knowledge of the theory of discrete-time dynamical systems, click here. We assume a basic familiarity with the theory and proceed with casting our diagram blocks as dynamical system blocks.

![Traditional feedback system diagram](../../../_static/tutorial/theory_to_python/cruise_control_as_a_composition_of_dynamical_systems.svg)

We discuss the derivation of each dynamical system block below.
:::{note} Although the signals between blocks have remained, for the most part, unchanged, the **summing junction** and **inverter** have dissapeared. Indeed, they've been absorbed into the PID block, as our implementation of the algorithm computes the error term internally.

### Block 1: Setpoint Speed

<div style="text-align: center;">
 <img src="../../../_static/tutorial/theory_to_python/cruise_control_block.svg"
      alt="Traditional feedback system diagram"
      style="max-width: 100%;">
</div>

Suppose we have a button on our steering wheel. When the button is pressed up, the setpoint speed is incremented; when the button is pressed down, the setpoint speed is decremented.

**State**: Setpoint speed $s_k$

**Inputs**: Button signal $b_k \in \{-1, 0, 1\}$ (decrement, neutral, increment)

**Evolution**:

$$
s_{k+1} = f(s_k, b_k) = s_k + b_k
$$

**Output**: Reference speed

$$
r_k = h(s_k) = s_k
$$

We now implement this system using pykal.

In [ ]:

import numpy as np
from pykal import DynamicalSystem


def setpoint_f(sk: float, bk: float) -> float:
    return sk + bk


def setpoint_h(sk: float) -> float:  # we dont need bk as an argument; it is not used
    return sk


setpoint_speed_generator = DynamicalSystem(f=setpoint_f, h=setpoint_h, state_name="sk")

We have just created a **DynamicalSystem** object. These objects contain all of the information we need about our dynamical system, as we see below:

In [ ]:

print(setpoint_speed_generator.__dict__)

Every DynamicalSystem object has three attributes:
- the **evolution function**, f
- the **output function**, h
- the **state name** (as a string).

Every DynamicalSystem object has only one method:
- ".step()" : compute the dynamical system one step forward

We demonstrate below stepping forward once:

In [ ]:
sk = 20  # initial setpoint speed
bk = 1  # pressing up on the button

ys = setpoint_speed_generator.step(param_dict={"sk": sk, "bk": bk})
print(ys)

Alternatively, we can return both the updated state and the output (it is often necessary to keep track of the state during simulation).

In [ ]:
sk_next, ys = setpoint_speed_generator.step(
    param_dict={"sk": sk, "bk": bk}, return_state=True
)
print(sk_next, ys)  # in this case, these are both the same anyway

Of course, we may step over a time period as well. Here, we simulate the following scenario: we press up on the button once a second for over 30 seconds.

In [ ]:
# Simulation time
dt = 1  # seconds
sim_time = np.arange(0, 30, dt)

# Initial Conditions
sk = 20
bk = 1

sk_hist = []  # keep history of set speed

for tk in sim_time:
    sk_hist.append(sk)
    sk_next, ys = setpoint_speed_generator.step(
        param_dict={"sk": sk, "bk": bk}, return_state=True
    )
    sk = sk_next

# plotting
from matplotlib import pyplot as plt

plt.plot(sim_time, sk_hist)
plt.xlabel("time (seconds)")
plt.ylabel("sk (setpoint speed)")
plt.title("Setpoint Speed Over Time")
plt.grid(True, alpha=0.3)
plt.show()

We are limited only by our creativity. Here, we simulate the following situation:

"We begin at an initial setpoint speed of 20 mph. After *five* seconds, we feel bold and slam up on the cruise control button. The button increases by *three* times a second. But our cruise control maxes out at *80* mph, and after *five* seconds at the maximum setting, we chicken out and press down on the cruise control, which decreases by *three* times a second, until our setpoint is back at 20 mph."

We first change our evolution function: it now maxes out at 80. We might as set a lower threshold of 0 while we're at it, since negative speed wouldn't make sense.

In [ ]:
def new_setpoint_f(sk: float, bk: float) -> float:
    return np.clip(sk + bk, 0, 80)


setpoint_speed_generator.f = new_setpoint_f


We now begin our new simulation. Note that there are *many* ways to implement the scenario above; the implementation below is the first that came to mind. We encourage you to experiment with changing several parameters and noting the change in the resulting graph.

In [ ]:
# Simulation time
dt = 1  # seconds
time_steps = np.arange(0, 60, dt)

button_rate = 3  # "presses" per second -> delta speed per second

# Initial state
sk = 20.0
sk_hist = []

button_mode = "neutral"  # neutral | ramp_up | hold_max | ramp_down | done
t_hit_max = None  # time when we first reached max


for tk in time_steps:
    sk_hist.append(sk)

    # --- Decide bk  ---
    if button_mode == "neutral":
        bk = 0.0
        if tk >= 5:
            button_mode = "ramp_up"

    if button_mode == "ramp_up":
        bk = +button_rate
        if sk >= 80.0:  # we reached/clipped to max
            button_mode = "hold_max"
            t_hit_max = tk

    elif button_mode == "hold_max":
        bk = 0.0
        # After 5 seconds at max, start ramping down
        if t_hit_max is not None and (tk - t_hit_max) >= 5:
            button_mode = "ramp_down"

    elif button_mode == "ramp_down":
        bk = -button_rate
        if sk <= 20.0:
            button_mode = "done"
            bk = 0.0

    elif button_mode == "done":
        bk = 0.0

    # --- Step the generator ---
    sk, ys = setpoint_speed_generator.step(
        param_dict={"sk": sk, "bk": bk}, return_state=True
    )

from matplotlib import pyplot as plt

plt.plot(time_steps, sk_hist)
plt.xlabel("time (seconds)")
plt.ylabel("sk (setpoint speed)")
plt.title("Setpoint Generator Simulation")
plt.grid(True, alpha=0.3)
plt.show()


This seems like a fun scenario to play with, so we will keep it and incorporate our other dynamical systems into it. For the sake of compositional convenience, we wrap the code above in a **callback** function (these will be used extensively in the ros implementation).

In [ ]:

# we initialize our callback using a closure (a function that returns a function)


def initialize_callback_with_parameters(
    button_mode: str = "neutral",
    *,
    button_rate: float = 3.0,
    t_start_ramp_up: float = 5.0,
    sk_min: float = 0.0,
    sk_max: float = 80.0,
    sk_target: float = 20.0,
    hold_seconds_at_max: float = 5.0,
):
    """
    Returns a stateful callback that implements:
      - neutral until t >= t_start_ramp_up
      - ramp_up at +button_rate until sk hits sk_max
      - hold at sk_max for hold_seconds_at_max
      - ramp_down at -button_rate until sk returns to sk_target
      - done (neutral) afterwards

    The callback signature is: callback(tk, *, sk) -> bk
    """
    t_hit_max = None  # captured state

    def callback(tk: float, *, sk: float) -> float:
        nonlocal button_mode, t_hit_max

        if button_mode == "neutral":
            if tk >= t_start_ramp_up:
                button_mode = "ramp_up"
            return 0.0

        if button_mode == "ramp_up":

            if sk >= sk_max:
                button_mode = "hold_max"
                t_hit_max = tk
                return 0.0
            return +button_rate

        if button_mode == "hold_max":
            if t_hit_max is None:
                t_hit_max = tk
            if (tk - t_hit_max) >= hold_seconds_at_max:
                button_mode = "ramp_down"
                return -button_rate
            return 0.0

        if button_mode == "ramp_down":
            if sk <= sk_target:
                button_mode = "done"
                return 0.0
            return -button_rate

        # done
        return 0.0

    return callback

Now we have a much cleaner implementation of our scenario, and we can play with the relevant parameters more easily.

In [ ]:
button_signal_callback = initialize_callback_with_parameters(
    button_mode="neutral",
    button_rate=3,
    t_start_ramp_up=5,
    sk_max=80,
    sk_target=20,
    hold_seconds_at_max=5,
)

# Simulation time
dt = 1  # seconds
time_steps = np.arange(0, 60, dt)

sk = 20.0
sk_hist = []

for tk in time_steps:
    sk_hist.append(sk)
    bk = button_signal_callback(tk, sk=sk)
    sk, ys = setpoint_speed_generator.step(
        param_dict={"sk": sk, "bk": bk}, return_state=True
    )

from matplotlib import pyplot as plt

plt.plot(time_steps, sk_hist)
plt.xlabel("time (seconds)")
plt.ylabel("sk (setpoint speed)")
plt.title("Setpoint Generator with Callback Policy")
plt.grid(True, alpha=0.3)
plt.show()

This concludes our dive into the dynamical system behind the cruise control button, as well as how to implement it in pykal and how to experiment with different scenarios. The remaining dynamical systems will be covered without such examples, as they can be found in the respective "Algorithm Library" notebooks.

### Block 2: PID

<div style="text-align: center;">
 <img src="../../../_static/tutorial/theory_to_python/controller_block.svg"
      style="max-width: 100%;">
</div>

The PID controller compares the setpoint to the estimated state and outputs a control command.

**Inputs**: Reference speed $r_k$, state estimate $\hat{x}_k$

**Outputs**: Control input $u_k$ (throttle command)

The discrete-time PID algorithm maintains an internal state tracking error history and computes proportional, integral, and derivative terms.

The implementation can be found in [PID](../../../src/pykal/controllers/pid.py).

In [ ]:
from pykal.algorithm_library.controllers import pid

controller = DynamicalSystem(f=pid.f, h=pid.h, state_name="ck")

### Block 3: Car

<div style="text-align: center;">
 <img src="../../../_static/tutorial/theory_to_python/plant_block.svg"
      alt="Traditional feedback system diagram"
      style="max-width: 100%;">
</div>

We model the car as a first-order system in velocity driven by the control input $u_k$ (throttle command).

**State**: Car velocity $x_k$

**Inputs**: Control input $u_k$ (throttle force in Newtons)

**Dynamics**: Linear drag model with sampling time $\Delta t$

$$
x_{k+1} = f_p(x_k, u_k) = x_k + \Delta t \left(-\frac{b}{m} x_k + \frac{1}{m} u_k\right)
$$

where $m > 0$ is the car mass (kg) and $b \geq 0$ is the drag coefficient (kg/s).

**Output**: Measured velocity (noiseless for now)

$$
y_k = h_p(x_k) = x_k
$$

We implement this discrete-time dynamical system below.

In [ ]:
from numpy.random import default_rng

rng = default_rng(seed=42)


def plant_f(xk: float, uk: float, m: float, b: float, dt: float) -> float:
    return xk + dt * (-b / m * xk + uk / m)


def plant_h(xk: float) -> float:
    return xk


plant = DynamicalSystem(f=plant_f, h=plant_h, state_name="xk")

### Block 4: KF

<div style="text-align: center;">
 <img src="../../../_static/tutorial/theory_to_python/observer_block.svg"
      style="max-width: 100%;">
</div>

The Kalman filter estimates the car's velocity by fusing the motion model with noisy measurements.

**Inputs**: Control input $u_k$, measurement $y_k$

**Outputs**: State estimate $\hat{x}_k$ and covariance $P_k$

The KF maintains an internal belief state $(\hat{x}_k, P_k)$ representing the estimated velocity and uncertainty. It performs a predict-update cycle using the car dynamics and measurement model.

The implementation can be found in [KF](../../../src/pykal/state_estimators/kf.py).

In [ ]:
from pykal.algorithm_library.estimators import kf

observer = DynamicalSystem(f=kf.f, h=kf.h, state_name="xhat_P")

## Simulation
![Traditional feedback system diagram](../../../_static/tutorial/theory_to_python/cruise_control_as_a_composition_of_dynamical_systems.svg)

We now simulate the complete closed-loop system, integrating all four dynamical components.

### System Parameters

In [ ]:
# Physical parameters
m = 1500.0  # Car mass (kg)
b = 50.0  # Drag coefficient (kg/s)
dt = 0.1  # Sampling time (s)

# PID gains (hand-tuned for this system)
KP = 800.0  # Proportional gain
KI = 50.0  # Integral gain
KD = 200.0  # Derivative gain

# Kalman filter parameters (1x1 matrices for scalar system)
Q = np.array([[0.1]])  # Process noise covariance
R = np.array([[1.0]])  # Measurement noise covariance

### Initial Conditions

In [ ]:
# Initial states
ck = (0.0, 0.0, 0.0)  # Controller: (error, integral, prev_error)
xk = 20.0  # Car starts at 20 mph
xhat = xk
xhat_P = (
    np.array([[xhat]]),
    np.array([[1.0]]),
)  # Observer: (estimate as 1x1 array, covariance as 1x1 array)
sk = xhat_P[0][0, 0]  # cruise control starts at current speed estimate

# Storage for plotting
time = []
setpoints = []
true_velocities = []
estimated_velocities = []
measurements = []
control_inputs = []
errors = []

### Simulate

In [ ]:
# Simulation time
T_sim = 30.0  # seconds
time_steps = np.arange(0, T_sim, dt)

# Initialize the button policy callback
policy = initialize_callback_with_parameters(
    button_mode="neutral",
    button_rate=3,
    t_start_ramp_up=5,
    sk_max=80,
    sk_target=20,
    hold_seconds_at_max=5,
)

# Run closed-loop simulation
for tk in time_steps:
    # Setpoint generator step (cruise control button)
    bk_button = policy(tk, sk=sk)
    sk, rk = setpoint_speed_generator.step(
        return_state=True, param_dict={"sk": sk, "bk": bk_button}
    )

    # Get current state estimate from observer (extract scalar from 1x1 array)
    xhat = observer.h(xhat_P)[0, 0]

    # PID controller step
    ck, bk = controller.step(
        return_state=True,
        param_dict={
            "ck": ck,
            "rk": rk,
            "xhat_k": xhat,
            "KP": KP,
            "KI": KI,
            "KD": KD,
        },
    )

    # Plant step
    xk, yk = plant.step(
        return_state=True,
        param_dict={"xk": xk, "uk": bk, "m": m, "b": b, "dt": dt},
    )

    # Observer step (Kalman Filter)
    # Compute Jacobians as 1x1 matrices for scalar system
    Fk = np.array([[1 - dt * b / m]])  # Jacobian: ∂f/∂xk
    Hk = np.array([[1.0]])  # Measurement Jacobian (linear measurement)

    xhat_P, xhat_obs = observer.step(
        return_state=True,
        param_dict={
            "xhat_P": xhat_P,
            "yk": np.array([[yk]]),  # Convert scalar measurement to 1x1 array
            "f": plant.f,
            "f_params": {"xk": xhat, "uk": bk, "m": m, "b": b, "dt": dt},
            "h": plant.h,
            "h_params": {"xk": xhat},
            "Fk": Fk,
            "Qk": Q,
            "Hk": Hk,
            "Rk": R,
        },
    )

    # Store results
    time.append(tk)
    setpoints.append(rk)
    true_velocities.append(xk)
    estimated_velocities.append(xhat)
    measurements.append(yk)
    control_inputs.append(bk)
    errors.append(rk - xhat)

### Visualize
We can visualize the pertinent values of our system to assure correct behaviour. In this case, note that the KF estimate is spot on, while the PID has an unnacceptable overshoot. Try tuning the PID in the loop above and see if you can get better performance.

In [ ]:
from matplotlib import pyplot as plt

fig, axs = plt.subplots(3, 1, figsize=(12, 10))

# Plot 1: Velocity Tracking
axs[0].plot(time, setpoints, "k--", label="Setpoint (r)", linewidth=2)
axs[0].plot(time, true_velocities, "b-", label="True Velocity (x)", alpha=0.7)
axs[0].plot(time, estimated_velocities, "r-", label="Estimated Velocity (x̂)", alpha=0.7)
axs[0].scatter(
    time[::10], measurements[::10], c="gray", s=10, alpha=0.3, label="Measurements (y)"
)
axs[0].set_ylabel("Velocity (mph)", fontsize=12)
axs[0].set_title(
    "Car Cruise Control: Velocity Tracking", fontsize=14, fontweight="bold"
)
axs[0].legend(loc="upper right")
axs[0].grid(True, alpha=0.3)

# Plot 2: Control Input
axs[1].plot(time, control_inputs, "g-", label="Control Input (u)", linewidth=1.5)
axs[1].set_ylabel("Control Input (N)", fontsize=12)
axs[1].set_title("Controller Output (Throttle Command)", fontsize=14, fontweight="bold")
axs[1].legend(loc="upper right")
axs[1].grid(True, alpha=0.3)

# Plot 3: Tracking Error
axs[2].plot(time, errors, "m-", label="Tracking Error (e = r - x̂)", linewidth=1.5)
axs[2].axhline(y=0, color="k", linestyle=":", alpha=0.5)
axs[2].set_xlabel("Time (s)", fontsize=12)
axs[2].set_ylabel("Error (mph)", fontsize=12)
axs[2].set_title("Tracking Error", fontsize=14, fontweight="bold")
axs[2].legend(loc="upper right")
axs[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Callback Wrapper

Just as we wrapped the setpoint generator in a callback function for compositional convenience, we can wrap the **entire cruise control system** in a single callback. This encapsulates all four dynamical systems (setpoint generator, PID controller, plant, and observer) and their states, making it easy to reuse and experiment with different configurations.

In [ ]:
def initialize_cruise_control_system(
    # Initial states
    sk_init: float = 20.0,
    ck_init: tuple = (0.0, 0.0, 0.0),
    xk_init: float = 20.0,
    xhat_init: float = 20.0,
    P_init: np.ndarray = None,
    # Physical parameters
    m: float = 1500.0,
    b: float = 50.0,
    dt: float = 0.1,
    # PID gains
    KP: float = 800.0,
    KI: float = 50.0,
    KD: float = 200.0,
    # Kalman filter parameters
    Q: np.ndarray = None,
    R: np.ndarray = None,
    # Button policy parameters
    button_mode: str = "neutral",
    button_rate: float = 3.0,
    t_start_ramp_up: float = 5.0,
    sk_min: float = 0.0,
    sk_max: float = 80.0,
    sk_target: float = 20.0,
    hold_seconds_at_max: float = 5.0,
):
    """
    Returns a stateful callback that encapsulates the entire cruise control system.

    The system consists of four dynamical systems:
      1. Setpoint generator (cruise control button)
      2. PID controller
      3. Plant (car dynamics)
      4. Observer (Kalman filter)

    The callback signature is: callback(tk) -> dict

    Returns a dictionary with keys:
      - 'setpoint': reference speed (rk)
      - 'true_velocity': actual car velocity (xk)
      - 'estimated_velocity': KF estimate (xhat)
      - 'measurement': noisy measurement (yk)
      - 'control_input': throttle command (uk)
      - 'error': tracking error (rk - xhat)
    """
    # Set default KF parameters if not provided
    if Q is None:
        Q = np.array([[0.1]])
    if R is None:
        R = np.array([[1.0]])
    if P_init is None:
        P_init = np.array([[1.0]])

    # Initialize internal states (captured by closure)
    sk = sk_init
    ck = ck_init
    xk = xk_init
    xhat_P = (np.array([[xhat_init]]), P_init)

    # Initialize button policy callback
    button_policy = initialize_callback_with_parameters(
        button_mode=button_mode,
        button_rate=button_rate,
        t_start_ramp_up=t_start_ramp_up,
        sk_min=sk_min,
        sk_max=sk_max,
        sk_target=sk_target,
        hold_seconds_at_max=hold_seconds_at_max,
    )

    def callback(tk: float) -> dict:
        nonlocal sk, ck, xk, xhat_P

        # 1. Setpoint generator step
        bk_button = button_policy(tk, sk=sk)
        sk, rk = setpoint_speed_generator.step(
            return_state=True, param_dict={"sk": sk, "bk": bk_button}
        )

        # 2. Extract current state estimate from observer
        xhat = observer.h(xhat_P)[0, 0]

        # 3. PID controller step
        ck, uk = controller.step(
            return_state=True,
            param_dict={
                "ck": ck,
                "rk": rk,
                "xhat_k": xhat,
                "KP": KP,
                "KI": KI,
                "KD": KD,
            },
        )

        # 4. Plant step
        xk, yk = plant.step(
            return_state=True,
            param_dict={"xk": xk, "uk": uk, "m": m, "b": b, "dt": dt},
        )

        # 5. Observer step (Kalman Filter)
        # Compute Jacobians as 1x1 matrices for scalar system
        Fk = np.array([[1 - dt * b / m]])  # Jacobian: ∂f/∂xk
        Hk = np.array([[1.0]])  # Measurement Jacobian (linear measurement)

        xhat_P, xhat_obs = observer.step(
            return_state=True,
            param_dict={
                "xhat_P": xhat_P,
                "yk": np.array([[yk]]),  # Convert scalar measurement to 1x1 array
                "f": plant.f,
                "f_params": {"xk": xhat, "uk": uk, "m": m, "b": b, "dt": dt},
                "h": plant.h,
                "h_params": {"xk": xhat},
                "Fk": Fk,
                "Qk": Q,
                "Hk": Hk,
                "Rk": R,
            },
        )

        # Return all relevant outputs (extract scalars for plotting)
        return {
            "setpoint": rk,
            "true_velocity": xk,
            "estimated_velocity": xhat,  # Use the extracted scalar, not xhat_obs
            "measurement": yk,
            "control_input": uk,
            "error": rk - xhat,
        }

    return callback

Now we can run the same simulation with a much cleaner interface. All system parameters can be configured at initialization, and the simulation loop becomes trivial:

In [ ]:
# Initialize the complete cruise control system
cruise_control = initialize_cruise_control_system(
    # Initial states
    sk_init=20.0,
    ck_init=(0.0, 0.0, 0.0),
    xk_init=20.0,
    xhat_init=20.0,
    # Physical parameters
    m=1500.0,
    b=50.0,
    dt=0.1,
    # PID gains
    KP=800.0,
    KI=50.0,
    KD=200.0,
    # Kalman filter parameters
    Q=np.array([[0.1]]),
    R=np.array([[1.0]]),
    # Button policy parameters
    button_mode="neutral",
    button_rate=3,
    t_start_ramp_up=5,
    sk_max=80,
    sk_target=20,
    hold_seconds_at_max=5,
)

# Simulation time
T_sim = 30.0
dt = 0.1
time_steps = np.arange(0, T_sim, dt)

# Storage for results
results = {
    "time": [],
    "setpoint": [],
    "true_velocity": [],
    "estimated_velocity": [],
    "measurement": [],
    "control_input": [],
    "error": [],
}

# Run simulation
for tk in time_steps:
    output = cruise_control(tk)

    results["time"].append(tk)
    results["setpoint"].append(output["setpoint"])
    results["true_velocity"].append(output["true_velocity"])
    results["estimated_velocity"].append(output["estimated_velocity"])
    results["measurement"].append(output["measurement"])
    results["control_input"].append(output["control_input"])
    results["error"].append(output["error"])

In [ ]:
# Visualize results
fig, axs = plt.subplots(3, 1, figsize=(12, 10))

# Plot 1: Velocity Tracking
axs[0].plot(
    results["time"], results["setpoint"], "k--", label="Setpoint (r)", linewidth=2
)
axs[0].plot(
    results["time"],
    results["true_velocity"],
    "b-",
    label="True Velocity (x)",
    alpha=0.7,
)
axs[0].plot(
    results["time"],
    results["estimated_velocity"],
    "r-",
    label="Estimated Velocity (x̂)",
    alpha=0.7,
)
axs[0].scatter(
    results["time"][::10],
    results["measurement"][::10],
    c="gray",
    s=10,
    alpha=0.3,
    label="Measurements (y)",
)
axs[0].set_ylabel("Velocity (mph)", fontsize=12)
axs[0].set_title(
    "Car Cruise Control: Velocity Tracking (Callback Interface)",
    fontsize=14,
    fontweight="bold",
)
axs[0].legend(loc="upper right")
axs[0].grid(True, alpha=0.3)

# Plot 2: Control Input
axs[1].plot(
    results["time"],
    results["control_input"],
    "g-",
    label="Control Input (u)",
    linewidth=1.5,
)
axs[1].set_ylabel("Control Input (N)", fontsize=12)
axs[1].set_title("Controller Output (Throttle Command)", fontsize=14, fontweight="bold")
axs[1].legend(loc="upper right")
axs[1].grid(True, alpha=0.3)

# Plot 3: Tracking Error
axs[2].plot(
    results["time"],
    results["error"],
    "m-",
    label="Tracking Error (e = r - x̂)",
    linewidth=1.5,
)
axs[2].axhline(y=0, color="k", linestyle=":", alpha=0.5)
axs[2].set_xlabel("Time (s)", fontsize=12)
axs[2].set_ylabel("Error (mph)", fontsize=12)
axs[2].set_title("Tracking Error", fontsize=14, fontweight="bold")
axs[2].legend(loc="upper right")
axs[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Experimentation

Now that we have a working cruise control system, try experimenting with different parameters:

**Controller tuning**:
- Adjust PID gains (KP, KI, KD) to reduce overshoot
- What happens if you increase KP too much? (oscillations)
- What happens if KD is too small? (slow settling)
- Find optimal gains for minimal overshoot and settling time

**Setpoint policy**:
- Change button policy parameters (button_rate, hold_seconds_at_max)
- Try different target speeds and ramp rates
- Design a smooth sinusoidal setpoint trajectory

**Plant variations**:
- Modify physical parameters (m, b) to simulate different vehicles
  - Heavy truck: m=5000 kg, b=100 kg/s
  - Sports car: m=1000 kg, b=30 kg/s
- Add measurement noise to make the problem more realistic
- Tune Kalman filter covariances (Q, R) for different noise scenarios

**Challenge**: Can you design PID gains that work well for both a heavy truck and a sports car?

[← Control Systems as Dynamical Systems](../../../getting_started/theory_to_python/control_systems_as_dynamical_systems.rst)